In [8]:
import pandas as pd

def is_a_buy_txn(txn):
    sum_buy = txn["sum_buy"]
    sum_sell = txn["sum_sell"]
    return sum_buy > sum_sell

def is_a_sell_txn(txn):
    return not is_a_buy_txn(txn)

def calc_sell_volume(txn):
    # if inventory decreased from one day to the next, consider it a sale and return the delta
    if txn["net_inventory"] < txn["prev_net_inventory"]:
        return txn["prev_net_inventory"] - txn["net_inventory"]
    
    # otherwise, assume we don't sell units
    return 0

def append_calcs_for_txns_for_date(txns_for_date):
    cumulative_net_inventory_for_day = 0

    for i, txn in txns_for_date.iterrows():
        if is_a_buy_txn(txn):
            txns_for_date.at[i, "net_inventory"] = cumulative_net_inventory_for_day + txn["sum_buy"]
            cumulative_net_inventory_for_day = txns_for_date.at[i, "net_inventory"]
            continue

        if is_a_sell_txn(txn):
            # we assume we start with 0 inventory
            # if the sale would yield negative inventory, sell 100% of inventory, and reset the net inventory back to 0
            # otherwise, we just decrement the net inventory by the amount we sold
            next_net_inventory = cumulative_net_inventory_for_day - txn["sum_sell"]
            if next_net_inventory < 0:
                txns_for_date.at[i, "net_inventory"] = 0
                cumulative_net_inventory_for_day = 0
            else:
                txns_for_date.at[i, "net_inventory"] = next_net_inventory
                cumulative_net_inventory_for_day = txns_for_date.at[i, "net_inventory"]
            continue

    txns_for_date["prev_net_inventory"] = txns_for_date["net_inventory"].shift(1)
    txns_for_date["sell_volume"] = txns_for_date.apply(calc_sell_volume, axis=1)

    txns_for_date["cost_of_buys"] = txns_for_date["sum_buy"] * txns_for_date["avg_price"]
    txns_for_date["cum_cost"] = txns_for_date["cost_of_buys"].cumsum()

    txns_for_date["revenue_from_sells"] = txns_for_date["sell_volume"] * txns_for_date["avg_price"]
    txns_for_date["cum_revenue"] = txns_for_date["revenue_from_sells"].cumsum()
    txns_for_date = txns_for_date.fillna(0)
    txns_for_date["profit"] = txns_for_date["cum_revenue"] - txns_for_date["cum_cost"]
    return txns_for_date

# read the file input.csv into a dataframe
input_df = pd.read_csv("./input.csv").fillna(0).assign(net_inventory=0)

working_df = pd.DataFrame(columns=input_df.columns)
for (date, txns_for_date) in input_df.groupby("date"):
    ret = append_calcs_for_txns_for_date(txns_for_date)
    working_df = pd.concat([working_df, ret])
    
# stripping out the intermediate columns added to the working dataframe
output_df = working_df[input_df.columns.append(pd.Index([ "profit",  "cum_cost", "cum_revenue", "profit" ]))]

# write the processed dataframe into the output file out.csv
output_df.to_csv("./out.csv")

from IPython.display import display, HTML

# input
display(HTML(input_df[input_df["date"] == "1/9/12"].to_html()))

# output
display(HTML(output_df[output_df["date"] == "1/9/12"].to_html()))

,date_and_time,date,Qualifiers,Instrument,trader_broker_id,aggressive_trader_class,interval_5min_index,ct_all,ct_buy,ct_sell,sum_volume,avg_volume,sum_buy,sum_sell,avg_price,total_cost,net_inventory
0,00:00.0,1/9/12,Si,WBC,945,high-frequency,132,44,0,44,31584,717.818182,0.0,31584.0,20.00,631680.00,0
1,05:00.0,1/9/12,Bi,WBC,945,high-frequency,133,2,2,0,8514,4257.000000,8514.0,0.0,19.99,170194.86,0
2,55:00.0,1/9/12,Bi,WBC,945,high-frequency,143,2,2,0,3532,1766.000000,3532.0,0.0,19.97,70534.04,0
3,45:00.0,1/9/12,Bi,WBC,945,high-frequency,153,6,6,0,16042,2673.666667,16042.0,0.0,19.99,320679.58,0
4,45:00.0,1/9/12,Bi,WBC,945,high-frequency,165,2,2,0,5512,2756.000000,5512.0,0.0,19.99,110184.88,0
5,55:00.0,1/9/12,Bi,WBC,945,high-frequency,179,6,6,0,17072,2845.333333,17072.0,0.0,19.96,340757.12,0
6,30:00.0,1/9/12,Si,WBC,945,high-frequency,186,12,0,12,3722,310.166667,0.0,3722.0,20.01,74477.22,0


,date_and_time,date,Qualifiers,Instrument,trader_broker_id,aggressive_trader_class,interval_5min_index,ct_all,ct_buy,ct_sell,sum_volume,avg_volume,sum_buy,sum_sell,avg_price,total_cost,net_inventory,profit,cum_cost,cum_revenue,profit
0,00:00.0,1/9/12,Si,WBC,945,high-frequency,132,44,0,44,31584,717.818182,0.0,31584.0,20.0,631680.0,0,0.00,0.00,0.00,0.00
1,05:00.0,1/9/12,Bi,WBC,945,high-frequency,133,2,2,0,8514,4257.0,8514.0,0.0,19.99,170194.86,8514,-170194.86,170194.86,0.00,-170194.86
2,55:00.0,1/9/12,Bi,WBC,945,high-frequency,143,2,2,0,3532,1766.0,3532.0,0.0,19.97,70534.04,12046,-240728.90,240728.90,0.00,-240728.90
3,45:00.0,1/9/12,Bi,WBC,945,high-frequency,153,6,6,0,16042,2673.666667,16042.0,0.0,19.99,320679.58,28088,-561408.48,561408.48,0.00,-561408.48
4,45:00.0,1/9/12,Bi,WBC,945,high-frequency,165,2,2,0,5512,2756.0,5512.0,0.0,19.99,110184.88,33600,-671593.36,671593.36,0.00,-671593.36
5,55:00.0,1/9/12,Bi,WBC,945,high-frequency,179,6,6,0,17072,2845.333333,17072.0,0.0,19.96,340757.12,50672,-1012350.48,1012350.48,0.00,-1012350.48
6,30:00.0,1/9/12,Si,WBC,945,high-frequency,186,12,0,12,3722,310.166667,0.0,3722.0,20.01,74477.22,46950,-937873.26,1012350.48,74477.22,-937873.26
